In [1]:
from kiteconnect import KiteConnect
from datetime import datetime,timedelta
import pandas as pd
import plotly.graph_objects as go

In [2]:
kite = KiteConnect(api_key="3np0d05xqq02k905")
kite.set_access_token("uS1z1psPljc3OOlgDQLeUjWMEifASGHZ")
# df_instruments=pd.DataFrame(kite.instruments())

In [3]:
def get_candle_fig(instrument_code,start,end,interval):
    df_historical=pd.DataFrame(kite.historical_data(instrument_code,start,end,interval))
    if len(df_historical)>0:
        fig = go.Figure(data=[go.Candlestick(x=df_historical['date'],
                        open=df_historical['open'],
                        high=df_historical['high'],
                        low=df_historical['low'],
                        close=df_historical['close'])])
        return (fig,df_historical)
    else:
        return (None,pd.DataFrame())
    
def get_candle(instrument_code,start,end,interval):
    df_historical=pd.DataFrame(kite.historical_data(instrument_code,start,end,interval))
    if len(df_historical)>0:
        return (df_historical)
    else:
        return (pd.DataFrame())

In [24]:
def get_candle_signal(instrument_code,sample_size):
    try:
        df_summary=pd.DataFrame()
        for i in range(sample_size):
            start=((datetime.now()+timedelta(days=-i)).strftime("%Y-%m-%d"))+" 09:00:00"
            end=((datetime.now()+timedelta(days=-i)).strftime("%Y-%m-%d"))+" 16:00:00"
            interval="5minute"
            df_historical=get_candle(instrument_code,start,end,interval)
            
            if len(df_historical)>0:
                df_historical['change']=df_historical['close']-df_historical['open']
                df_historical['candle_type']=df_historical['change'].apply(lambda x:'green' if x>0 else 'red')
                try:
                    candle_ratio= ((df_historical['candle_type'].value_counts()['green'])/(df_historical['candle_type'].count()))
                except:
                    candle_ratio=0
                df_summary_dict={'day':end,
                                'candle_ratio':candle_ratio}
                df_temp=pd.DataFrame([df_summary_dict])
                df_summary=pd.concat([df_summary,df_temp])
        return ((sum(df_summary['candle_ratio'])*2)/sample_size)
    except:
        return 0
    
def get_candle_signal_2(instrument_code,sample_size):
    try:
        df_summary=pd.DataFrame()
        for i in range(sample_size):
            start=((datetime.now()+timedelta(days=-i)).strftime("%Y-%m-%d"))+" 09:00:00"
            end=((datetime.now()+timedelta(days=-i)).strftime("%Y-%m-%d"))+" 16:00:00"
            interval="5minute"
            df_historical=get_candle(instrument_code,start,end,interval)
            
            if len(df_historical)>0:
                df_historical['change']=df_historical['close']-df_historical['open']
                df_historical['candle_type']=df_historical['change'].apply(lambda x:'green' if x>0 else 'red')
                try:
                    candle_ratio= ((df_historical['candle_type'].value_counts()['green'])/(df_historical['candle_type'].count()))
                except:
                    candle_ratio=0
                df_summary_dict={'day':end,
                                'candle_ratio':candle_ratio}
                df_temp=pd.DataFrame([df_summary_dict])
                df_summary=pd.concat([df_summary,df_temp])
        return (len(df_summary[df_summary['candle_ratio']>0.5])/len(df_summary))
    except:
        return 0
    
def get_candle_signal_3(instrument_code,sample_size):
    try:
        df_summary=pd.DataFrame()
        for i in range(sample_size):
            start=((datetime.now()+timedelta(days=-i)).strftime("%Y-%m-%d"))+" 09:00:00"
            end=((datetime.now()+timedelta(days=-i)).strftime("%Y-%m-%d"))+" 16:00:00"
            interval="5minute"
            df_historical=get_candle(instrument_code,start,end,interval)
            
            if len(df_historical)>0:
                df_historical['change']=df_historical['close']-df_historical['open']
                df_historical['candle_type']=df_historical['change'].apply(lambda x:1 if x>0 else -1)
                df_historical['candle_weight']=df_historical['change'].abs()/df_historical['change'].abs().max()
                df_historical['candle_strength']=df_historical['candle_type']*df_historical['candle_weight']
                try:
                    candle_ratio= sum(df_historical['candle_strength'])
                except:
                    candle_ratio=0
                df_summary_dict={'day':end,
                                'candle_ratio':candle_ratio}
                df_temp=pd.DataFrame([df_summary_dict])
                df_summary=pd.concat([df_summary,df_temp])
        return ((sum(df_summary['candle_ratio']))/sample_size)
    except:
        return 0

In [9]:
df_instruments1=pd.read_excel('final_instruments.xlsx')
df_instruments1['signal_1']=df_instruments1['instrument_token'].apply(lambda x: get_candle_signal(x,31))
df_instruments1['signal_2']=df_instruments1['instrument_token'].apply(lambda x: get_candle_signal_2(x,31))

In [16]:
df_instruments1['signal_1_7d']=df_instruments1['instrument_token'].apply(lambda x: get_candle_signal(x,7))
df_instruments1['signal_2_7d']=df_instruments1['instrument_token'].apply(lambda x: get_candle_signal_2(x,7))

In [25]:
df_instruments1['signal_3_7d']=df_instruments1['instrument_token'].apply(lambda x: get_candle_signal_3(x,7))


In [28]:
df_instruments1['signal_3_31d']=df_instruments1['instrument_token'].apply(lambda x: get_candle_signal_3(x,31))

In [30]:
df_instruments1.sort_values(by="signal_3_31d",ascending=False).reset_index(drop=True).to_csv('weighted_final_shortlist.csv',index=False)

In [19]:
sample_size=7
instrument_code=2865921
df_summary=pd.DataFrame()
for i in range(sample_size):
    start=((datetime.now()+timedelta(days=-i)).strftime("%Y-%m-%d"))+" 09:00:00"
    end=((datetime.now()+timedelta(days=-i)).strftime("%Y-%m-%d"))+" 16:00:00"
    interval="5minute"
    df_historical=get_candle(instrument_code,start,end,interval)
    
    if len(df_historical)>0:
        df_historical['change']=df_historical['close']-df_historical['open']
        df_historical['candle_type']=df_historical['change'].apply(lambda x:1 if x>0 else -1)
        df_historical['candle_weight']=df_historical['change'].abs()/df_historical['change'].abs().max()
        df_historical['candle_strength']=df_historical['candle_type']*df_historical['candle_weight']
        try:
            candle_ratio= ((df_historical['candle_type'].value_counts()['green'])/(df_historical['candle_type'].count()))
        except:
            candle_ratio=0
        df_summary_dict={'day':end,
                        'candle_ratio':candle_ratio}
        df_temp=pd.DataFrame([df_summary_dict])
        df_summary=pd.concat([df_summary,df_temp])

In [23]:
dff=df_historical.sort_values(by='change')
sum(dff['candle_strength'])

-0.48453608247427693

In [ ]:

# Place an order
order_id = kite.place_order(tradingsymbol="INFY",
                            exchange=kite.EXCHANGE_NSE,
                            transaction_type=kite.TRANSACTION_TYPE_BUY,
                            quantity=1,
                            variety=kite.VARIETY_AMO,
                            order_type=kite.ORDER_TYPE_MARKET,
                            product=kite.PRODUCT_CNC,
                            validity=kite.VALIDITY_DAY)



# Fetch all orders
kite.orders()

# Get instruments
kite.instruments()

# Place an mutual fund order
kite.place_mf_order(
    tradingsymbol="INF090I01239",
    transaction_type=kite.TRANSACTION_TYPE_BUY,
    amount=5000,
    tag="mytag"
)

# Cancel a mutual fund order
kite.cancel_mf_order(order_id="order_id")

# Get mutual fund instruments
kite.mf_instruments()

In [18]:
a=-1
a.abs()

AttributeError: 'int' object has no attribute 'abs'